In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from sklearn import linear_model

# Any results you write to the current directory are saved as output.

In [32]:
# 将各值变成数值
data_train = pd.read_csv('../input/train.csv')
data_test = pd.read_csv('../input/test.csv')
def cabin(df):
    df.loc[(df.Cabin.notnull()), 'Cabin'] = "Yes"
    df.loc[(df.Cabin.isnull()), 'Cabin'] = "No"
    return df

def process(data_train):
    dummies_cabin = pd.get_dummies(data_train['Cabin'], prefix = 'Cabin')
    dummies_pclass = pd.get_dummies(data_train['Pclass'], prefix = 'Pclass')
    dummies_sex = pd.get_dummies(data_train['Sex'], prefix = 'Sex')   
    df = pd.concat([data_train, dummies_pclass, dummies_sex, dummies_cabin], axis=1)
    
    # 暂时先丢弃age
    df.drop(['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'PassengerId', 'Age'], axis=1, inplace=True)
    return df
data_train = cabin(data_train)
df = process(data_train)

train_df = df.filter(regex='Survived|Cabin_.*|SibSp|Parch|Sex_.*|Pclass_.*')
train_np = train_df.as_matrix()

# y为Survival结果
y = train_np[:, 0]

# x为特征属性值
X = train_np[:, 1:]

clf = linear_model.LogisticRegression(C=1.0, penalty='l1', tol=1e-6)
clf.fit(X, y)

In [33]:
data_test = cabin(data_test)
df_test = process(data_test)
test = df_test.filter(regex='Cabin_.*|SibSp|Parch|Sex_.*|Pclass_.*')
predictions = clf.predict(test)

result = pd.DataFrame({'PassengerId':data_test['PassengerId'].as_matrix(), 'Survived':predictions.astype(np.int32)})
result.to_csv('prediction.csv', index=False)

pd.DataFrame({"columns":list(train_df.columns)[1:], "coef":list(clf.coef_.T)})